In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name= "unsloth/Meta-Llama-3.1-8B", 
    max_seq_length= max_seq_length, 
    dtype= dtype, 
    load_in_4bit= load_in_4bit,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r= 16, 
    target_modules= ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha= 16, 
    lora_dropout= 0, 
    bias= "none", 
    use_gradient_checkpointing= "unsloth", 
    random_state= 3407, 
    use_rslora= False,
    loftq_config= None,
)
#lora adapter, we set the weights as W + AB, where W is the original weights and A and B are new vectors we add. 
#W stays the same from trained model, but A and B are adjusted in training. Since W is a x by y matrix, then A is a x by 1 and B is a 1 by x vector.
#This is significantly less weight adjusting compared to retraining the model, making it much faster
#-> only really trainined a tenth of the data

In [ ]:
import pandas as pd

df = pd.read_csv('discord_messages.csv')
df.tail() 

In [ ]:
EOS_TOKEN = tokenizer.eos_token
print(EOS_TOKEN)

In [ ]:
def formatting_prompt(examples):
  input = examples['input']
  output = examples['output']
  texts = []
  for input, output in zip(input, output):
    text = alpaca_prompt.format(input, output) + EOS_TOKEN
    texts.append(text)

  return { "text": texts, }
pass

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model= model, 
    tokenizer= tokenizer, 
    train_dataset= dataset, 
    dataset_text_field= 'text', 
    max_seq_length= max_seq_length, 
    dataset_num_proc= 2, 
    packing= False,
    args= TrainingArguments(
        per_device_train_batch_size= 2, 
        gradient_accumulation_steps= 4, 
        warmup_steps= 5, 
        max_steps= 60, 
        learning_rate= 2e-4, 
        fp16= not is_bfloat16_supported(), 
        bf16= is_bfloat16_supported(), 
        logging_steps= 1, 
        optim= 'adamw_8bit', 
        weight_decay= 0.01, 
        lr_scheduler_type= 'linear',
        seed= 3407, 
        output_dir= 'outputs', 
    ), 
)

In [ ]:
model.save_pretrained_gguf('model', tokenizer, quantization_method='f16')

In [ ]:
%%shell
cd model
ls
cat <<EOF > Modelfile
FROM ./unsloth.F16.gguf

SYSTEM You are a chatbot that responds to the input with the appropriate output.
EOF

In [ ]:
%%shell
ollama serve
ollama create llama3chat -f ./Modelfile